In [1]:
import numpy as np
import pandas as pd
import gym_bandits
import gym
from collections import Counter
env = gym.make("BanditTenArmedGaussian-v0")

WARN: Environment '<class 'gym_bandits.bandit.BanditTenArmedGaussian'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


In [2]:
class ucb_LearningAgent:
    def __init__(self, n_bandits):
        self.n_bandits = n_bandits
        self.Q_all = np.random.rand(n_bandits)
        self.N_all = np.ones(n_bandits) # intialize number of selections for all bandits as ones
        self.ucb_all = np.array([self.Q_all[i] + np.sqrt(2*(np.log(1)/self.N_all[i])) for i in range(n_bandits)])
    def update(self, observation, reward, action, t):
        self.N_all[action]+=1 # increment number of selections for selected bandit (action)
        self.Q_all[action]+=(reward - self.Q_all[action])/self.N_all[action] # update Q with the formula. Average so far
        self.ucb_all = np.array([self.Q_all[i] + np.sqrt(2*(np.log(t)/self.N_all[i])) for i in range(10)])
    def next_action(self,observation):
        return self.ucb_all.argmax()

In [3]:
ucb_agent = ucb_LearningAgent(10)

In [4]:
ucb_records = []
for t in range(1,1000):
    observation = env.reset()
    action = ucb_agent.next_action(observation)
    observation, reward, done, info = env.step(action)
    ucb_agent.update(observation,reward,action,t)
    ucb_records.append([t, action, observation, reward])
ucb_df = pd.DataFrame(ucb_records,columns=["Time","Action","Observation","Reward"])

In [5]:
ucb_df['Reward'].sum()

1000.2883183512706